In [1]:
import gradio as gr
import importlib
import re
import os
import pickle
from IPython.display import Markdown
from python_scripts import llm_rag, machine_translation, text_to_speech, whisper_setup, get_audio, utils
import numpy as np
from scipy.io.wavfile import write
import librosa

# r8_eYbLLc9wPQ1qa4dUK8eXyewFKHvGAck1h9LZl

# Define paths dynamically
PATH = os.getcwd().replace('\\\\', '/')

with open('symptom_list.pkl', 'rb') as f:
    symptom_list = pickle.load(f)

root_path = PATH + '\\Datasets\\MeDAL'
audio_path = PATH + '\\Datasets\\Audio_Files'

In [2]:

# importlib.reload(llm_rag)
# importlib.reload(machine_translation)
# importlib.reload(text_to_speech)
# importlib.reload(whisper_setup)
# importlib.reload(get_audio)
# importlib.reload(utils)


In [3]:
def SMTS(Query):
    try:
        # Process the audio input
        file_path = 'output.wav'
        write(file_path, data = np.array(Query[1], dtype = np.int16), rate = Query[0])
        audio_processed = utils.preprocess_audio(file_path)

        # Transcribe Query to English
        whisper_models = ["tiny", "base", "small", "medium", "large"]
        
        transcript = whisper_setup.transcribe_audio(audio_processed, ['tiny'])
        text = (transcript['tiny'][2]).lower()
        print(text)

        # Regular expression pattern to match symptoms containing 'or' any symptoms from the list
        pattern = r'\b(?:' + '|'.join(map(re.escape, symptom_list)) + \
        '|'.join('(?:{}|{})'.format(re.escape(symptom.split(' or ')[0]), re.escape(symptom.split(' or ')[1])) \
                 for symptom in symptom_list if ' or ' in symptom) + r')\b'

        # Extract symptoms from the query
        extracted_symptoms = re.findall(pattern, text, flags = re.IGNORECASE)
        print(extracted_symptoms)

        # Feed query into the LLM
        models = { 
        'llama_ours': 'ubaidtariq8/llama2-med-genai', # fine tuned model
        'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
        'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
        'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
        }

        model = llm_rag.DocumentEmbeddingPipeline(model_version = models['mixtral'], chroma_path = root_path)
        model.setup_environment()
        model.prepare_documents(collection_name = "muqeem", joining = True, persistent = True)
        model.embed_and_index()
        query = ('You are a medical doctor. A patient has come to you for desperate need of help. ' +
         'Give as accurate diagnosis as possible tinyd on the symptoms listed. ' +
         ' '.join(extracted_symptoms) + '. Also consider the whole query ' + text + ' ' +
         'Give also suggestions for mitigating the problem.')


        response = model.query_data(query + ' ' + transcript['tiny'][2])
        
        # Translate it back to the user language
        translated_text = machine_translation.translate_text(text = response.response, src_lang = 'en', trg_lang = transcript['tiny'][0])
        print(translated_text)
        # Step 5: Now speak the response in the user's language
        audio_answer_path = audio_path + '/audio.wav'
        text_to_speech.multilingual_text_to_speech(text = translated_text, filepath = audio_answer_path)
        utils.sasti_harkat(audio_answer_path)
        arr, sr = librosa.load(audio_answer_path)
        # display(Markdown(f"<b>{translated_text}</b>"))
        return text, translated_text, (sr, arr)
    except Exception as e:
        print("An error occurred:", e)

In [7]:
# Launch the Gradio Interface
demo = gr.Interface(
    fn = SMTS,
    inputs = [gr.Audio(label = 'Get your Voice Heard! 🔍', sources = ["microphone"])],
    outputs = [gr.Textbox(label = "We have heard your Voice! 👂"), gr.Textbox(label = "This is what we recommend 📋"), gr.Audio(label = 'Press Play to Listen to your Mecial Report 🔊')],
    allow_flagging = 'never',
    theme = 'gradio/base',
    title = 'SymptoCare: Your Personalized Healthcare Assistant 🤖',
    description = '''## Welcome to SymptoCare! 🌟
    Discover the power of seamless communication in healthcare with SymptoCare, your personalized healthcare translator!
    ### How It Works:
    1. 🎤 Speak your symptoms.
    2. 🔄 Let SymptoCare translate them into actionable insights.
    3. 🗨️ Engage with your healthcare provider like never before!''',

    article = '''### What We Offer:
    - 🗣️ Break language barriers with ease.
    - 📲 Translate your symptoms into accurate diagnoses.
    - 🤝 Empower your healthcare journey with personalized care.

    ### Join Us Today:
    Get started now and take control of your healthcare journey! Check our [Github](https://github.com/CS-5302/CS-5302-Project-Group-15) here! Do give us a star if you like our work! 😀'''
)

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python

 i have fever, headache and difficulty breathing.
['fever', 'headache', 'difficulty breathing']


Loading files: 100%|██████████| 2/2 [00:00<00:00, 55.45file/s]


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

...

 i have fever, headache and difficulty breathing.
['fever', 'headache', 'difficulty breathing']


Loading files: 100%|██████████| 2/2 [00:00<00:00, 45.58file/s]


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/71 [00:00<?, ?it/s]

Based on the symptoms you've described, it's possible that you could be experiencing a respiratory infection, such as the flu or pneumonia. These conditions often present with fever, headache, and difficulty breathing, among other symptoms. However, it's important to note that this is just a possibility and not a definitive diagnosis. 

I would strongly recommend that you seek immediate medical attention. These symptoms could potentially indicate a more serious condition, such as COVID-19 or a severe case of the flu. A healthcare professional will be able to provide a more accurate diagnosis and appropriate treatment.

In the meantime, there are a few things you can do to help alleviate your symptoms:

1. Rest and drink plenty of fluids to stay hydrated.
2. Over-the-counter medications, such as acetaminophen or ibuprofen, can help reduce fever and alleviate headache and body aches.
3. If you're having trouble breathing, use a humidifier to add moisture to the air, which can help soothe